# transformer
attention 由由bengio团队于2014年提出， 并在近年来得到广泛的应用。
transformer中抛弃了传统CNN和RNN, 整个网络结构完全式由attention机制组成。准确来讲，transfoerm 由且仅由self-attetion 和feed forward neural network 组成。一个基于transformer的形式进行搭建。

对于RNN等类型的网络，其计算是顺序的，这样就存在两个问题
1）时间片t的计算依赖t-1时刻的计算结果，泽洋限制了模型的并行能力
2）顺序计算的过程中，信息会丢失，尽管有了LSTM的门机制来缓解长期依赖的问题，但是对于特别长期的信息就无能为力了。

transfomer针对这两个问题，采取了相应的措施。首先使用attenion机制，将序列中的任意两个位置之间的距离缩小为一个常量；其次它不是类似RNN的顺序结构，因此具有更好的并行性。

transformer 本质上是一个encoder decoder 的结构，那么
![title](img/transformer_v1.png)

如论文中所设置的，编码器由六个编码block组成，解码器器同样由6个解码器block组成，与所有的生成模型相同的是，编码器的输出会作为解码器的输入。如下图所示：
![title](img/transformer_v2.png)

对于encoder 模型， 数据首先会经过一个叫做“self-attention”的模块得到一个加权之后的特征向量Z, 

$$Attention(Q,K,V)=softmax(\frac{QK^T}{\sqrt{d_k}})V$$

得到Z之后，它会被送到encoder的下一个模块， 即feed forward neural network, 也就是全连接层。该网络一共两层，第一层激活啊哈桑农户是Relu, 第二层是象形激活函数，可以表示为：

$$FFN(Z)=max(0, ZW_1+b_1)W_2+b_2$$

decoder 结构如下图所示，他和encoder的不同之处在于decoder多了一个encoder-decoder attention , 两个attention分别用于计算输入和输出的权值。

1 self-attention: 当前翻译和已经翻译的前文之间的关系
2 encoder-decoder attention: 当前翻译和编码的特征向量之间的关系。

![title](img/transformer_v3.png)

输入编码

首先使用word2vec 将词转化为向量，在最底层的block中，$x$将直接作为transformer的输入，而在其他层中，输入则是上一个block的输出。

![title](img/transformer_v4.png)

self-attention

self-attention 是 transformer最核心的内容。在self-attention中，每个单词有3个不同的向量，它们分别是Query 向量（Q）, key向量（K）和value 向量（V）, 长度均是64. 它们是通过3个不同的权值矩阵由嵌入向量乘以三个不同的权值矩阵$W^Q$, $W^K$, $W^V$得到， 其中三个矩阵的尺寸也是相同的。

那么Query\Key\value的意义是啥？

- 将输入单词转化为嵌入向量
- 根据嵌入向量得到q, k, v三个向量
- 为每个向量计算一个score $score=q.k$
- 为了梯度的稳定，transformer 使用了score归一化， 即除以$\sqrt{d_k}$
- 对score施以softmax 激活函数
- softmax点乘value值v, 得到加权的每个输入向量的评分v
- 相加之后得到最终的输出结果z: $z=\sum{v}$

self-attention 最后一点采用了残差网络的short-cut结构，解决网络退化问题。

![title](img/transformer_v5.png)

## Multi-Head attention
multi-head attention 相当于h个不同的self-attention的集成， 在这里我们以h=8举例说明。

- 根据X分别输入8个self-attention中，得到8个加权后的特征矩阵， $Z_i,i\in{i,2,...,8}$
- 将8个$Z_i按列拼成一个大的特征矩阵$
- 特征矩阵经过一层全连接后得到输出Z

过程如下图所示
![titile](img/transformer_v6.png)

同时这里也加入了short-cut机制

## Encoder-Decoder Attention


在解码器中，Transformer block 比编码器中多了个encoder-decoder attention. 在encoder decoder attention 中， Q来自于解码器的上一个输出，K和V则来自于编码器的输出，其计算方式完全和上面的计算方式相同。

## 损失层

解码器解码之后，解码的特征向量经过一层激活函数为softmax的全连接层之后得到反映每个单词概率的概率输出向量。我们可以通过CTC等损失函数训练模型。

一个完整可训练的网络结构便是encoder和decoder的堆叠（各N个）。我们可以得到完整的transformer的结构。

![title](img/transformer_v7.png)

## 位置编码

transformer模型对顺序序列的处理能力，也就是说无论句子的结构怎么达伦，transformer都会得到类似的结果。

怎么解决位置信息？
a 根据数据学习
b 自己设计编码规则。

原论文给出公式

$$PE(pos,2i)=sin(\frac{pos}{10000^{\frac{2i}{d_model}}})$$
$$PE(pos,2i+1)=cos(\frac{pos}{10000^{2i}{d_model}})$$

在上面的式子中，pos表示单词的位置，i表示单词的维度，

根据公式$sin(\alpha+\beta)=sin{\alpha}cos{\beta}+cos{\alpha}sin{\beta}$以及$cos(\alpha+\beta)=cos{\alpha}cos{\beta}-sin{\alpha}sin{\beta}$， 这表明位置k+p的位置向量可以表示为位置k的特征向量的线性变化。


In [1]:
# t 是

In [6]:
import numpy as np
import tensorflow as tf

In [3]:
def get_angles(pos, i, d_model):
    # 这里的i等价与上面公式中的2i和2i+1
    angle_rates = 1 / np.power(10000, (2*(i // 2))/ np.float32(d_model))
    return pos * angle_rates

In [4]:
def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                           np.arange(d_model)[np.newaxis,:],
                           d_model)
    # 第2i项使用sin
    sines = np.sin(angle_rads[:, 0::2])
    # 第2i+1项使用cos
    cones = np.cos(angle_rads[:, 1::2])
    pos_encoding = np.concatenate([sines, cones], axis=-1)
    pos_encoding = pos_encoding[np.newaxis, ...]
    
    return tf.cast(pos_encoding, dtype=tf.float32)

In [8]:
pos_encoding = positional_encoding(10, 20)
print(pos_encoding.shape)

(1, 10, 20)


In [9]:
pos_encoding

<tf.Tensor: id=3, shape=(1, 10, 20), dtype=float32, numpy=
array([[[ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
          0.0000000e+00,  0.0000000e+00,  1.0000000e+00,  1.0000000e+00,
          1.0000000e+00,  1.0000000e+00,  1.0000000e+00,  1.0000000e+00,
          1.0000000e+00,  1.0000000e+00,  1.0000000e+00,  1.0000000e+00],
        [ 8.4147096e-01,  3.8767424e-01,  1.5782665e-01,  6.3053876e-02,
          2.5116222e-02,  9.9998331e-03,  3.9810613e-03,  1.5848925e-03,
          6.3095731e-04,  2.5118864e-04,  5.4030228e-01,  9.2179644e-01,
          9.8746681e-01,  9.9801010e-01,  9.9968451e-01,  9.9994999e-01,
          9.9999207e-01,  9.9999875e-01,  9.9999982e-01,  9.9999994e-01],
        [ 9.0929741e-01,  7.1471345e-01,  3.1169716e-01,  1.2585682e-01,
          5.0216600e-02,  1.9998666e-02,  7.9620592e-03,  3.1697811e-03,
          1.2619144e-03,  5.0237728e-04, -4.1614684e-01,  6.994